In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import re
import time


In [2]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# 

In [3]:
torch.manual_seed(1337)

In [4]:
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
# Split the text into words                    
words = re.findall(r'\S+|\n|\n\n', text)                    

# Create vocabulary
vocab = sorted(set(words))
vocab_size = len(vocab)

# Create mappings
stoi = {word: i for i, word in enumerate(vocab)}
itos = {i: word for i, word in enumerate(vocab)}

# Update encode and decode functions
encode = lambda s: [stoi[word] for word in re.findall(r'\S+|\n|\n\n', s)]
decode = lambda l: ' '.join([itos[i] for i in l])


print(f'Some words in vocabulary: {vocab[10000:20000]}')
print(f'vocab_size: {vocab_size}')

Some words in vocabulary: ['dull;', 'dullest', 'dulness,', 'duly', 'dumb', 'dumb,', 'dumbly', 'dumbness,', 'dump', 'dump,', 'dumps', 'dumps?', "dun's", 'dun,', 'dungeon.', 'dungy', 'durance,', 'durance?', 'durst', 'dusky', 'dust', 'dust,', 'dust.', 'dust?', 'duteous', 'duties,', 'duty', 'duty!', "duty's", 'duty,', 'duty.', 'duty:', 'duty;', 'duty?', 'dwell', 'dwellest', 'dwelling', 'dwelling,', 'dwells', 'dwells,--which', 'dwells;', 'dyed', 'dying', 'dying,', "e'en", "e'er", 'each', 'each,', 'eager', 'eagle', "eagle's", "eagle's,", 'eagle,', 'eagle-winged', 'eagles', 'eagles.', 'ean:', 'ear', 'ear,', 'ear-deafening', 'ear.', 'ear:', 'ear;', 'earl', 'earl.', 'earldom', 'earldom,--', 'earliness', 'earls', 'earls,', 'early', 'early:', 'early?', 'earn', "earn'd", 'earned', 'earnest', 'earnest,', 'earnest.', 'earnest:', 'earnest?', 'earnestly', 'earnestness', 'ears', 'ears!', 'ears,', 'ears--waving', 'ears.', 'ears:', 'ears;', 'ears?', 'earth', 'earth!', "earth's", 'earth,', 'earth.', 'eart

In [6]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [7]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        perplexities = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
            perplexities[k] = torch.exp(loss).item()
        # out[split] = losses.mean()
        out[split + '_loss'] = losses.mean()
        out[split + '_perplexity'] = perplexities.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [9]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

3.512903 M parameters


In [10]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

start_time = time.time()
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train_loss']:.4f}, val loss {losses['val_loss']:.4f}, train perplexity {losses['train_perplexity']:.4f}, val perplexity {losses['val_perplexity']:.4f}")


    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

end_time = time.time()


step 0: train loss 10.3121, val loss 10.3052, train perplexity 30105.0781, val perplexity 29899.4004
step 100: train loss 6.9401, val loss 7.0703, train perplexity 1044.9847, val perplexity 1188.6161
step 200: train loss 6.6762, val loss 6.8859, train perplexity 801.7634, val perplexity 986.8019
step 300: train loss 6.5194, val loss 6.7898, train perplexity 685.6995, val perplexity 897.6638
step 400: train loss 6.3993, val loss 6.7156, train perplexity 607.8617, val perplexity 835.0300
step 500: train loss 6.2491, val loss 6.6373, train perplexity 525.0513, val perplexity 773.8726
step 600: train loss 6.1856, val loss 6.6344, train perplexity 492.9517, val perplexity 773.3282
step 700: train loss 6.0911, val loss 6.5748, train perplexity 448.6282, val perplexity 728.8820
step 800: train loss 5.9756, val loss 6.5583, train perplexity 399.7852, val perplexity 717.2770
step 900: train loss 5.9047, val loss 6.5198, train perplexity 372.0274, val perplexity 689.9285
step 1000: train loss 5.

In [11]:
training_time = end_time - start_time

In [12]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


 ESCALUS: 
 Stand you go; ten thousand counsel? 
 
 POMPEY: 
 Henry is a sir? I am, to stand a 
 crooked and hanged after. if I be so 
 
 CAMILLO: 
 He's so still; 'tis as he should see, for I may 
 To have your husband's licence: to go, if 
 And there but to have suppler with the pace 
 Where I do not enough to have tripp'd thus; 
 Ratcliff, we have the heavens, my morn Claudio? What 
 
 Page: 
 Younger you cannot, sir; 'Do he knows, to-morrow, and his 
 which 'longs hath his body to a minister 
 Such goodness to me, in his God, 
 Into a triumph mother doth full fresh treason: on't. 
 
 Third Citizen: 
 I would have well pleased spider, Catesby, 
 The my heart is gave idle? 
 My love I will not service before no words: 
 Shalt the supreme man's garden with my misery, 
 The untimely sister of prison do set the crown 
 And so it careful spoil look in thy brother. 
 Watching breeds leanness, leanness is drown'd untrodden 
 Above Bohemia's entire, why be seen; 
 For truth in revenue. woe

In [13]:
def generate_sample(model, seed_text, max_new_tokens=100):
    context = torch.tensor([encode(seed_text)], dtype=torch.long, device=device)
    return decode(model.generate(context, max_new_tokens=max_new_tokens)[0].tolist())

sample_text = generate_sample(model, "The quick brown fox")

In [14]:
print("Training Time: {training_time}")
print(sample_text)

Training Time: {training_time}
The quick brown fox 
 
 MENENIUS: 
 And, like grief must I the man 
 To trust me this I may potch like crooked 
 Charge me? Gentle as the trier 
 That I took the truth to my hand and holy; 
 And yet not sued on me not his report. 
 But, have put you may; yet he was a 
 And made him already, whispering, he nor understand with him. 
 
 DUKE OF AUMERLE: 
 I cannot tell the very well. 
 
 EDWARD: 
 My son, George me quarrel, thy constant fools. 
 
 NORTHUMBERLAND: 
 Thy castle
